In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
import re
from drop_parse_file_id import parse_drop_file_id
from occ import predict_y_offset

from glitch.impedance import EISSpectrumDoc

INPUT_FOLDER = "/Users/andreakowal/Downloads/"
OUTPUT_FOLDER = "/Users/andreakowal/Coding/Raw Nyquist Data" # where the raw Nyquist data points are being saved
OUTPUT_INTERCEPTS = "/Users/andreakowal/Coding/Intercept Data" # where the x-intercept values are being saved

# data folder path
data_folder = Path(INPUT_FOLDER + "Drop Experiments/FAS-50/NaCl/0.05 M").expanduser() 

# background correction files
OC_path = Path(INPUT_FOLDER + "Drop Experiments/Open Circuit Files/02_07_25_OCTest_C01.mpt").expanduser()
SC_path = Path(INPUT_FOLDER + "Drop Experiments/Short Circuit Files/07_23_25_SCTest_C01.mpt").expanduser()
oc = EISSpectrumDoc.from_eclab_mpt(OC_path)
sc = EISSpectrumDoc.from_eclab_mpt(SC_path)

# sort membrane files by membrane ID and replicate number
def extract_membrane_replicate_dropconc(file_path):
    membrane_ID = re.search(r"_(\d+)_", file_path.name)
    replicate = re.search(r"_R(\d+)_", file_path.name)
    drop_conc = re.search(r"_D([\d\.]+)M_", file_path.name)

    membrane_number = int(membrane_ID.group(1)) if membrane_ID else 999
    replicate_number = int(replicate.group(1)) if replicate else 999
    drop_conc_value = float(drop_conc.group(1)) if drop_conc else 999

    return (membrane_number, replicate_number, drop_conc_value)

nyquist_results = []

#Create dictionary for membrane thicknesses
membrane_id_thickness = {
    "01": 0.066,
    "02": 0.064,
    "03": 0.068,
}

for file_path in sorted(data_folder.glob("*.mpt"), key=extract_membrane_replicate_dropconc):
    print(f"Processing {file_path.name}")

    my_spectrum = EISSpectrumDoc.from_eclab_mpt(file_path)
    my_spectrum.background_correct(oc.cycles_raw[0], sc.cycles_raw[0])

    #parse file name for meta data
    id_string = file_path.stem
    meta_data = parse_drop_file_id(id_string)

    #Make sure thickness corresponds to membrane ID
    membrane_id_key = str(meta_data["Membrane ID"]).zfill(2)
    thickness = membrane_id_thickness[membrane_id_key]

    #Get y offset from OCC code
    y_offset_occ = predict_y_offset(thickness)

    #Creating cleaner title for graphs
    graph_title = f"{meta_data['Soak Concentration (M)']} {meta_data['Salt']}"

    # Build subtitles on graphs that contain meta data
    subtitle_parts = []

    if "Drop Concentration (M)" in meta_data:
        subtitle_parts.append(f"Drop Concentration (M): {meta_data['Drop Concentration (M)']}")

    subtitle_parts.append(f"Membrane: {meta_data['Membrane']}")
    subtitle_parts.append(f"ID: {meta_data['Membrane ID']}")
    subtitle_parts.append(f"Replicate: {meta_data['Replicate']}")
    subtitle_parts.append(f"Date: {meta_data["Date"]}")

    subtitle = " | ".join(subtitle_parts)
    
    combined_title = f"{graph_title}<br><span style='font-size:14px'>{subtitle}</span>"

    #plot data for all loops
    fig = go.Figure()

    #create empty lists to store values
    all_reals = []
    all_imags = []
    all_freqs = []

    #go through loops in data file 
    for i, cycle in enumerate(my_spectrum.cycles_raw):
        frequency_mask = cycle.frequencies <= 100_000 #filtering out high frequencies
        freqs_filtered = cycle.frequencies[frequency_mask]
        impedance_filtered = cycle.impedance[frequency_mask]
        Z_real = impedance_filtered.real
        Z_imag = impedance_filtered.imag

        if i == 0:
            all_freqs = freqs_filtered #store frequenices

        all_reals.append(Z_real)
        all_imags.append(Z_imag)

        #Apply y-offset
        y_shifted = Z_imag - y_offset_occ
        #Delete all points that fall below the x-axis
        postive_mask = y_shifted >= 0

        fig.add_trace(go.Scatter(
                x=Z_real[postive_mask],
                y=y_shifted[postive_mask],
                mode='markers',
                name=f'Loop {i+1}'))

    #Average points across loops
    real_array = np.vstack(all_reals)
    imags_array = np.vstack(all_imags)

    average_real = np.mean(real_array, axis = 0)
    average_imaginary = np.mean(imags_array, axis = 0)

    #Shift OCC by y-offset
    y_average_shifted = average_imaginary - y_offset_occ
    #Delete all points that fall below the x-axis
    average_positive_mask = y_average_shifted >= 0
    x_vals = average_real[average_positive_mask]
    y_vals = y_average_shifted[average_positive_mask]

    # Save averaged (filtered) data to CSV
    df_avg = pd.DataFrame({
    "Frequency (Hz)": all_freqs[average_positive_mask],
    "Z_real (Ohm)": x_vals,
    "Z_imag (Ohm)": y_vals})
    
    avg_filename = Path(OUTPUT_FOLDER) / f"{file_path.stem}_average_filtered_points.csv"
    df_avg.to_csv(avg_filename, index=False)

    #Add averaged points to plot
    fig.add_trace(go.Scatter(
        x = x_vals,
        y = y_vals,
        mode = 'markers',
        name = 'Average',
        marker = dict(color="black", size = 6, symbol = "circle")))
    
    #Fit linear trendline through all shifted points
    if len(x_vals)>= 2:
        trendline = np.polyfit(x_vals, y_vals, deg=1)
        slope, intercept = trendline
        all_frequency_x_intercept = -intercept/slope

        #Add trendline to plot
        x_fit = np.linspace(min(x_vals.min(), all_frequency_x_intercept), max(x_vals.max(), all_frequency_x_intercept), 200)
        y_fit = slope * x_fit + intercept
        fig.add_trace(go.Scatter(
            x = x_fit,
            y = y_fit, 
            mode = "lines",
            name = "All Frequencies Trendline",
            line = dict(color = "black", dash = "dash", width = 2)))
    else: print("Not enough points to fit linear line")

    #Fit a linear trendling through shifted high frequency points
    high_frequency_mask = (all_freqs > 1000) & average_positive_mask
    x_high_frequency = average_real[high_frequency_mask]
    y_high_frequency = y_average_shifted[high_frequency_mask]

    if len(x_high_frequency) >= 2:
        high_frequency_trendline = np.polyfit(x_high_frequency, y_high_frequency, deg = 1)
        slope_high_frequency, intercept_high_frequency = high_frequency_trendline
        high_frequency_x_intercept = -intercept_high_frequency/slope_high_frequency

        #Add high frequency trendline to plot
        x_fit_high = np.linspace(min(x_high_frequency.min(), high_frequency_x_intercept), max(x_high_frequency.max(), high_frequency_x_intercept), 200)
        y_fit_high = slope_high_frequency * x_fit_high + intercept_high_frequency

        fig.add_trace(go.Scatter(
            x = x_fit_high,
            y = y_fit_high, 
            mode = "lines", 
            name = "High Frequency Trendline (> 1 kHZ)",
            line = dict(color = "red", dash = "dash", width = 2)))
        
    else: print("Not enough points to fit linear trendline")
    #Find x-value at the minimum averaged imaginary part
    #min_imag = np.argmin(y_average_shifted)
    #x_at_min_imag = average_real[min_imag]
    #x_intercept = x_at_min_imag

    fig.update_layout(
        title=combined_title,
        xaxis_title='Re{Z} (Ohm)',
        yaxis_title='-Im{Z} (Ohm)',
        xaxis=dict(scaleanchor="y", scaleratio=1),
        width=700,
        height=600)

    fig.show()

    nyquist_results.append({
        "Membrane ID": meta_data.get("Membrane ID"),
        "Replicate": meta_data.get("Replicate"),
        "Soak Concentration (M)": meta_data.get("Soak Concentration (M)"),
        "Drop Concentration (M)": meta_data.get("Drop Concentration (M)"),
        "Real intercept (all frequencies)":all_frequency_x_intercept,
        "Real intercept (high frequenices)": high_frequency_x_intercept})

df = pd.DataFrame(nyquist_results)

df = pd.DataFrame(nyquist_results)
df["Drop Concentration Numeric"] = df["Drop Concentration (M)"].str.replace(" M", "", regex=False).astype(float)

df_sorted = df.sort_values(by=["Membrane ID", "Replicate", "Drop Concentration Numeric"], ascending=[True, True, True])

df_sorted = df_sorted.drop(columns=["Drop Concentration Numeric"])

summary_file = f"/EIS_summary_{meta_data['Soak Concentration (M)'].replace(' ', '')}_{meta_data['Salt'].replace(' ', '_')}.csv"
df_sorted.to_csv(OUTPUT_INTERCEPTS + summary_file, index=False)
print(f"Saved summary file to: {OUTPUT_INTERCEPTS + summary_file}")
print(df_sorted)


Processing 0p05M_NaCl_D1M_01_R1_FAS50_20250722_01_GEIS_C01.mpt


Processing 0p05M_NaCl_D2M_01_R1_FAS50_20250723_01_GEIS_C01.mpt


Processing 0p05M_NaCl_D0p5M_01_R1_FAS50_20250722_01_GEIS_C01.mpt


Processing 0p05M_NaCl_D0p25M_01_R1_FAS50_20250721_01_GEIS_C01.mpt


Processing 0p05M_NaCl_D0p1M_01_R1_FAS50_20250721_01_GEIS_C01.mpt


...

Processing 0p05M_NaCl_D2M_03_R1_FAS50_20250723_01_GEIS_C01.mpt


Processing 0p05M_NaCl_D0p5M_03_R1_FAS50_20250722_01_GEIS_C01.mpt


Processing 0p05M_NaCl_D0p25M_03_R1_FAS50_20250721_01_GEIS_C01.mpt


Processing 0p05M_NaCl_D0p1M_03_R1_FAS50_20250721_01_GEIS_C01.mpt


Saved summary file to: /Users/andreakowal/Coding/Intercept Data/EIS_summary_0.05M_NaCl.csv
   Membrane ID Replicate Soak Concentration (M) Drop Concentration (M)  \
4           01         1                 0.05 M                  0.1 M   
3           01         1                 0.05 M                 0.25 M   
2           01         1                 0.05 M                  0.5 M   
0           01         1                 0.05 M                    1 M   
1           01         1                 0.05 M                    2 M   
9           02         1                 0.05 M                  0.1 M   
7           02         1                 0.05 M                 0.25 M   
8           02         1                 0.05 M                  0.5 M   
5           02         1                 0.05 M                    1 M   
6           02         1                 0.05 M                    2 M   
14          03         1                 0.05 M                  0.1 M   
13          03       